In [1]:
import pandas as pd
import numpy as np
from scipy.interpolate import CubicSpline, UnivariateSpline, InterpolatedUnivariateSpline, interp1d
from datetime import datetime
from math import floor
import matplotlib.pyplot as plt
from BEE_WEATHER_DATA import BROODMINDER_GET, AMBIENT_GET, READ_HIVE, PROCESS_HIVE, READ_BEE_WEATHER, PROCESS_BEE_WEATHER, PROCESS_AMBIENT, GRAPH_DATA, GET_MOON_IMAGE, GET_FORECAST, PROCESS_FORECAST

hive_name = "New Left Hive"
key = 1

if key == 1:
    Data = PROCESS_AMBIENT()
    display(Data)
    #GRAPH_DATA(Data)
if key == 2:
    Data = PROCESS_BEE_WEATHER()
    #display(Data)
    GRAPH_DATA(Data)
if key == 3:
    Data = PROCESS_HIVE(hive_name, interp=1)
    #display(Data)
    [GRAPH_DATA(Data[str(i)]) for i in Data.keys()]
if key == 4:
    pix = GET_MOON_IMAGE(216, save=1)
    plt.imshow(pix)
    plt.show()
if key == 5:
    data = GET_FORECAST()
    response = PROCESS_FORECAST(data)
    display(response)

print("Done")

,dateutc,winddir,windspeedmph,windgustmph,maxdailygust,tempf,humidity,hourlyrainin,eventrainin,dailyrainin,weeklyrainin,monthlyrainin,yearlyrainin,totalrainin,uv,solarradiation,feelsLike,dewPoint,lastRain
8063,1.654214e+09,298,0.00,1.12,13.65,75.2,73,0.0,0.00,0.00,0.19,0.00,2.93,2.93,0,41.59,75.85,65.94,2022-05-31T00:29:00.000Z
8064,1.654215e+09,350,0.67,1.12,13.65,75.0,73,0.0,0.00,0.00,0.19,0.00,2.93,2.93,0,35.48,75.63,65.74,2022-05-31T00:29:00.000Z
8065,1.654215e+09,347,0.00,0.00,13.65,74.8,74,0.0,0.00,0.00,0.19,0.00,2.93,2.93,0,30.34,75.46,65.94,2022-05-31T00:29:00.000Z
8066,1.654215e+09,3,0.45,1.12,13.65,74.7,74,0.0,0.00,0.00,0.19,0.00,2.93,2.93,0,22.51,75.35,65.85,2022-05-31T00:29:00.000Z
8067,1.654216e+09,224,1.12,4.47,13.65,74.5,75,0.0,0.00,0.00,0.19,0.00,2.93,2.93,0,18.11,75.18,66.04,2022-05-31T00:29:00.000Z
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10074,1.654818e+09,338,0.22,2.24,11.41,79.5,60,0.0,0.09,0.04,0.61,0.61,3.54,3.54,0,44.65,79.97,64.38,2022-06-09T08:41:00.000Z
10075,1.654818e+09,1,0.67,2.24,11.41,79.3,60,0.0,0.09,0.04,0.61,0.61,3.54,3.54,0,41.10,79.75,64.19,2022-06-09T08:41:00.000Z
10076,1.654819e+09,343,0.22,5.82,11.41,79.0,59,0.0,0.09,0.04,0.61,0.61,3.54,3.54,0,37.68,79.37,63.43,2022-06-09T08:41:00.000Z
10077,1.654819e+09,15,1.79,6.93,11.41,78.8,61,0.0,0.09,0.04,0.61,0.61,3.54,3.54,0,32.42,79.25,64.19,2022-06-09T08:41:00.000Z


Done


In [69]:
from datetime import datetime, date, timedelta, timezone
import pandas as pd
from math import floor
import time
from BEE_WEATHER_DATA import BROODMINDER_GET, AMBIENT_GET, READ_HIVE, PROCESS_HIVE, READ_BEE_WEATHER, PROCESS_BEE_WEATHER, PROCESS_AMBIENT, GRAPH_DATA, GET_MOON_IMAGE, GET_FORECAST, PROCESS_FORECAST
import pytz
import astral, astral.sun

def GET_WEATHER_ICON():
    lat = 40.907220
    lon = -111.894300
    tz = pytz.timezone('America/Denver')
    tz_name = 'America/Denver'
    for_date = date.today()
    
    l = astral.LocationInfo('Custom Name', 'My Region', tz_name, lat, lon)
    s = astral.sun.sun(l.observer, date=for_date)
    sunrise = s['sunrise'].astimezone(tz)
    sunset = s['sunset'].astimezone(tz)
    
    response = PROCESS_FORECAST()
    icon_dict = {200: "11d",
                201: "11d",
                202: "11d",
                210: "11d",
                211: "11d",
                212: "11d",
                221: "11d",
                230: "11d",
                231: "11d",
                232: "11d",
                300: "09d",
                301: "09d",
                302: "09d",
                310: "09d",
                311: "09d",
                312: "09d",
                313: "09d",
                314: "09d",
                321: "09d",
                500: "10d",
                501: "10d",
                502: "10d",
                503: "10d",
                504: "10d",
                511: "13d",
                520: "09d",
                521: "09d",
                522: "09d",
                531: "09d",
                600: "13d",
                601: "13d",
                602: "13d",
                611: "13d",
                612: "13d",
                613: "13d",
                615: "13d",
                616: "13d",
                620: "13d",
                621: "13d",
                622: "13d",
                701: "50d",
                711: "50d",
                721: "50d",
                731: "50d",
                741: "50d",
                751: "50d",
                761: "50d",
                762: "50d",
                771: "50d",
                781: "50d",
                "800d": "01d",
                "800n": "01n",
                "801d": "02d",
                "801n": "02n",
                "802d": "03d",
                "802n": "03n",
                "803d": "04d",
                "803n": "04n",
                "804d": "04d",
                "804n": "04n",
                }
    
    icon_url = []
    
    for idx, id in response["id"].items():
        if floor(id/100) == 8 and idx >= int(datetime.timestamp(sunrise)) and idx <= int(datetime.timestamp(sunset)):
            icon_url.append("https://openweathermap.org/img/wn/%s@2x.png" % (icon_dict[str(id) + "d"]))
        elif floor(id/100) == 8 and idx < int(datetime.timestamp(sunrise)) or idx > int(datetime.timestamp(sunset)):
            icon_url.append("https://openweathermap.org/img/wn/%s@2x.png" % (icon_dict[str(id) + "n"]))
        else:
            icon_url.append("https://openweathermap.org/img/wn/%s@2x.png" % (icon_dict[str(id)]))
    return icon_url

print(GET_WEATHER_ICON())

['https://openweathermap.org/img/wn/04n@2x.png', 'https://openweathermap.org/img/wn/04n@2x.png', 'https://openweathermap.org/img/wn/04n@2x.png', 'https://openweathermap.org/img/wn/03n@2x.png', 'https://openweathermap.org/img/wn/03n@2x.png', 'https://openweathermap.org/img/wn/04n@2x.png', 'https://openweathermap.org/img/wn/04n@2x.png', 'https://openweathermap.org/img/wn/02n@2x.png', 'https://openweathermap.org/img/wn/03n@2x.png', 'https://openweathermap.org/img/wn/04n@2x.png', 'https://openweathermap.org/img/wn/03n@2x.png', 'https://openweathermap.org/img/wn/01n@2x.png', 'https://openweathermap.org/img/wn/01n@2x.png', 'https://openweathermap.org/img/wn/01n@2x.png', 'https://openweathermap.org/img/wn/01n@2x.png', 'https://openweathermap.org/img/wn/01n@2x.png', 'https://openweathermap.org/img/wn/01n@2x.png', 'https://openweathermap.org/img/wn/01n@2x.png', 'https://openweathermap.org/img/wn/01n@2x.png', 'https://openweathermap.org/img/wn/04n@2x.png', 'https://openweathermap.org/img/wn/04n@